# Introduction

Discovering new code words in declassified CIA documents may seem like a mission for a foreign intelligence service, and detecting [gender biases](https://medium.com/agatha-codes/a-bossy-sort-of-voice-3c3a18de3093) in the Harry Potter novels a task for a literature professor. Yet by utilizing natural language parsing with regular expressions, the power to perform such analyses is in your own hands!

While you may not put much explicit thought into the structure of your sentences as you write, the syntax choices you make are critical in ensuring your writing has meaning. Analyzing such sentence structure as well as word choice can not only provide insights into the connotation of a piece text, but can also highlight the biases of its author or uncover additional insights that even a [deep, rigorous reading of the text might not reveal.](https://twitter.com/hexadecim8/status/1068215227274137605)

By using Python’s regular expression modulere and the Natural Language Toolkit, known as NLTK, you can find keywords of interest, discover where and how often they are used, and discern the parts-of-speech patterns in which they appear to understand the sometimes hidden meaning in a piece of writing. Let’s get started!



In [13]:
# Create a Chunk Counter
from collections import Counter

def np_chunk_counter(chunked_sentences):

    # create a list to hold chunks
    chunks = list()

    # for-loop through each chunked sentence to extract noun phrase chunks
    for chunked_sentence in chunked_sentences:
        for subtree in chunked_sentence.subtrees(filter=lambda t: t.label() == 'NP'):
            chunks.append(tuple(subtree))

    # create a Counter object
    chunk_counter = Counter()

    # for-loop through the list of chunks
    for chunk in chunks:
        # increase counter of specific chunk by 1
        chunk_counter[chunk] += 1

    # return 30 most frequent chunks
    return chunk_counter.most_common(30)

In [14]:
# Chunking

from nltk import RegexpParser
from pos_tagged_oz import pos_tagged_oz

# define noun-phrase chunk grammar here
chunk_grammar = "NP: {<DT>?<JJ>*<NN>}"

# create RegexpParser object here
chunk_parser = RegexpParser(chunk_grammar)

# create a list to hold noun-phrase chunked sentences
np_chunked_oz = list()

# create a for-loop through each pos-tagged sentence in pos_tagged_oz here
for pos_tagged_sentence in pos_tagged_oz:
  # chunk each sentence and append to np_chunked_oz here
  np_chunked_oz.append(chunk_parser.parse(pos_tagged_sentence))

# store and print the most common np-chunks here
most_common_np_chunks = np_chunk_counter(np_chunked_oz)
print(most_common_np_chunks)

[((('i', 'NN'),), 326), ((('dorothy', 'NN'),), 222), ((('the', 'DT'), ('scarecrow', 'NN')), 213), ((('the', 'DT'), ('lion', 'NN')), 148), ((('the', 'DT'), ('tin', 'NN')), 123), ((('woodman', 'NN'),), 112), ((('oz', 'NN'),), 86), ((('toto', 'NN'),), 73), ((('head', 'NN'),), 59), ((('the', 'DT'), ('woodman', 'NN')), 59), ((('the', 'DT'), ('wicked', 'JJ'), ('witch', 'NN')), 58), ((('the', 'DT'), ('emerald', 'JJ'), ('city', 'NN')), 51), ((('the', 'DT'), ('witch', 'NN')), 49), ((('the', 'DT'), ('girl', 'NN')), 46), ((('the', 'DT'), ('road', 'NN')), 41), ((('room', 'NN'),), 29), ((('nothing', 'NN'),), 29), ((('the', 'DT'), ('air', 'NN')), 29), ((('the', 'DT'), ('country', 'NN')), 26), ((('the', 'DT'), ('land', 'NN')), 24), ((('a', 'DT'), ('heart', 'NN')), 24), ((('the', 'DT'), ('west', 'NN')), 23), ((('axe', 'NN'),), 23), ((('the', 'DT'), ('sun', 'NN')), 22), ((('the', 'DT'), ('little', 'JJ'), ('girl', 'NN')), 22), ((('course', 'NN'),), 22), ((('the', 'DT'), ('cowardly', 'JJ'), ('lion', 'NN'

### 1 - Compiling and Matching

Before you dive into more complex syntax parsing, you’ll begin with basic regular expressions in Python using the re module as a regex refresher.

The first method you will explore is .compile(). This method takes a regular expression pattern as an argument and compiles the pattern into a regular expression object, which you can later use to find matching text. The regular expression object below will exactly match 4 upper or lower case characters.

regular_expression_object = re.compile("[A-Za-z]{4}")

Regular expression objects have a .match() method that takes a string of text as an argument and looks for a single match to the regular expression that starts at the beginning of the string. To see if your regular expression matches the string "Toto" you can do the following:

result = regular_expression_object.match("Toto")

If .match() finds a match that starts at the beginning of the string, it will return a match object. The match object lets you know what piece of text the regular expression matched, and at what index the match begins and ends. If there is no match, .match() will return None.

With the match object stored in result, you can access the matched text by calling result.group(0). If you use a regex containing capture groups, you can access these groups by calling .group() with the appropriately numbered capture group as an argument.

Instead of compiling the regular expression first and then looking for a match in separate lines of code, you can simplify your match to one line:

result = re.match("[A-Za-z]{4}","Toto")

With this syntax, re‘s .match() method takes a regular expression pattern as the first argument and a string as the second argument.

In [4]:
# 1 - Compiling and Matching

import re

# characters are defined
character_1 = "Dorothy"
character_2 = "Henry"

# compile your regular expression here
regular_expression = re.compile("\w{7}")

# check for a match to character_1 here
result_1 = regular_expression.match(character_1)
print(result_1)

# store and print the matched text here
match_1 = result_1.group(0)
print(match_1)

# compile a regular expression to match a 7 character string of word characters and check for a match to character_2 here
result_2 = re.match("\w{7}",character_2)
print(result_2)

<_sre.SRE_Match object; span=(0, 7), match='Dorothy'>
Dorothy
None


### 2 - Searching and Finding

You can make your regular expression matches even more dynamic with the help of the .search() method. Unlike .match() which will only find matches at the start of a string, .search() will look left to right through an entire piece of text and return a match object for the first match to the regular expression given. If no match is found, .search() will return None. For example, to search for a sequence of 8 word characters in the string Are you a Munchkin?:

result = re.search("\w{8}","Are you a Munchkin?")

Using .search() on the string above will find a match of "Munchkin", while using .match() on the same string would return None!

So far you have used methods that only return one piece of matching text. What if you want to find all the occurrences of a word or keyword in a piece of text to determine a frequency count? Step in the .findall() method!

Given a regular expression as its first argument and a string as its second argument, .findall() will return a list of all non-overlapping matches of the regular expression in the string. Consider the below piece of text:

text = "Everything is green here, while in the country of the Munchkins blue was the favorite color. But the people do not seem to be as friendly as the Munchkins, and I'm afraid we shall be unable to find a place to pass the night."

To find all non-overlapping sequences of 8 word characters in the sentence you can do the following:

list_of_matches = re.findall("\w{8}",text)

.findall() will thus return the list ['Everythi', 'Munchkin', 'favorite', 'friendly', 'Munchkin'].

In [16]:
# Searching and Finding

import re
from sherlock_holmes import bohemia_ch1

# import L. Frank Baum's The Wonderful Wizard of Oz
#oz_text = open("the_wizard_of_oz_text.txt",encoding='utf-8').read().lower()

oz_text = bohemia_ch1

# search oz_text for an occurrence of 'Baker' here
found_Baker = re.search("Baker", oz_text)
print(found_Baker)

# find all the occurrences of 'Holmes' in oz_text here
all_Holmes = re.findall("Holmes", oz_text)
print(all_Holmes)

# store and print the length of all_lions here
number_Holmes = len(all_Holmes)
print(number_Holmes)

print('-'*30)
# To Inspect a Function, use

import inspect
print(inspect.getsource(np_chunk_counter))

<_sre.SRE_Match object; span=(1528, 1533), match='Baker'>
['Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes', 'Holmes']
24
------------------------------
def np_chunk_counter(chunked_sentences):

    # create a list to hold chunks
    chunks = list()

    # for-loop through each chunked sentence to extract noun phrase chunks
    for chunked_sentence in chunked_sentences:
        for subtree in chunked_sentence.subtrees(filter=lambda t: t.label() == 'NP'):
            chunks.append(tuple(subtree))

    # create a Counter object
    chunk_counter = Counter()

    # for-loop through the list of chunks
    for chunk in chunks:
        # increase counter of specific chunk by 1
        chunk_counter[chunk] += 1

    # return 30 most frequent chunks
    return chunk_counter.most_common(30)



### 3 - Part-of-Speech Tagging

While it is useful to match and search for patterns of individual characters in a text, you can often find more meaning by analyzing text on a word-by-word basis, focusing on the part of speech of each word in a sentence. This process of identifying and labeling the part of speech of words is known as part-of-speech tagging!

It may have been a while since you’ve been in English class, so let’s review the nine parts of speech with an example:

Wow! Ramona and her class are happily studying the new textbook she has on NLP.

- Noun: the name of a person (Ramona,class), place, thing (textbook), or idea (NLP)
- Pronoun: a word used in place of a noun (her,she)
- Determiner: a word that introduces, or “determines”, a noun (the)
- Verb: expresses action (studying) or being (are,has)
- Adjective: modifies or describes a noun or pronoun (new)
- Adverb: modifies or describes a verb, an adjective, or another adverb (happily)
- Preposition: a word placed before a noun or pronoun to form a phrase modifying another word in the sentence (on)
- Conjunction: a word that joins words, phrases, or clauses (and)
- Interjection: a word used to express emotion (Wow)

You can automate the part-of-speech tagging process with nltk‘s pos_tag() function! The function takes one argument, a list of words in the order they appear in a sentence, and returns a list of tuples, where the first entry in the tuple is a word and the second is the part-of-speech tag.

Given the sentence split into a list of words below:

word_sentence = ['do', 'you', 'suppose', 'oz', 'could', 'give', 'me', 'a', 'heart', '?']<br>

you can tag the parts of speech as follows:

part_of_speech_tagged_sentence = pos_tag(word_sentence) <br>
The call to pos_tag() will return the following:

[('do', 'VB'), ('you', 'PRP'), ('suppose', 'VB'), ('oz', 'NNS'), ('could', 'MD'), ('give', 'VB'), ('me', 'PRP'), ('a', 'DT'), ('heart', 'NN'), ('?', '.')]

Abbreviations are given instead of the full part of speech name. Some common abbreviations include: NN for nouns, VB for verbs, RB for adverbs, JJ for adjectives, and DT for determiners. [A complete list of part-of-speech tags and their abbreviations can be found here.](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html)

In [9]:
# 4 - POS Tagginf

import nltk
from nltk import pos_tag
from word_tokenized_oz import word_tokenized_oz # Already Sentence Tokenized

# save and print the sentence stored at index 100 in word_tokenized_oz here

witches_fate = word_tokenized_oz[100]
print(witches_fate)
# ['``', 'the', 'house', 'must', 'have', 'fallen', 'on', 'her', '.']

# create a list to hold part-of-speech tagged sentences here
pos_tagged_oz  = list()

# create a for loop through each word tokenized sentence in word_tokenized_oz here
for text in word_tokenized_oz:
  # part-of-speech tag each sentence and append to pos_tagged_oz here
  pos_tagged_oz.append(pos_tag(text))

# store and print the 101st part-of-speech tagged sentence here
witches_fate_pos = pos_tagged_oz[100]
# [('``', '``'), ('the', 'DT'), ('house', 'NN'), ('must', 'MD'), ('have', 'VB'), ('fallen', 'VBN'), ('on', 'IN'), ('her', 'PRP'), ('.', '.')]

print(witches_fate_pos)

['``', 'the', 'house', 'must', 'have', 'fallen', 'on', 'her', '.']
[('``', '``'), ('the', 'DT'), ('house', 'NN'), ('must', 'MD'), ('have', 'VB'), ('fallen', 'VBN'), ('on', 'IN'), ('her', 'PRP'), ('.', '.')]
